In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

import datetime
import json
import time
import glob
import sys
import os
import gc

from sklearn import linear_model
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# import nltk
# nltk.download('movie_reviews')

In [3]:
from nltk.corpus import movie_reviews

In [4]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [5]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

In [6]:
movie_reviews.categories()

['neg', 'pos']

In [7]:
documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]

In [39]:
docs = [list(movie_reviews.words(fileid)) for fileid in movie_reviews.fileids()]

SyntaxError: invalid syntax (<ipython-input-39-f5f049b852a1>, line 1)

In [38]:
docs[:1]

[['plot',
  ':',
  'two',
  'teen',
  'couples',
  'go',
  'to',
  'a',
  'church',
  'party',
  ',',
  'drink',
  'and',
  'then',
  'drive',
  '.',
  'they',
  'get',
  'into',
  'an',
  'accident',
  '.',
  'one',
  'of',
  'the',
  'guys',
  'dies',
  ',',
  'but',
  'his',
  'girlfriend',
  'continues',
  'to',
  'see',
  'him',
  'in',
  'her',
  'life',
  ',',
  'and',
  'has',
  'nightmares',
  '.',
  'what',
  "'",
  's',
  'the',
  'deal',
  '?',
  'watch',
  'the',
  'movie',
  'and',
  '"',
  'sorta',
  '"',
  'find',
  'out',
  '.',
  '.',
  '.',
  'critique',
  ':',
  'a',
  'mind',
  '-',
  'fuck',
  'movie',
  'for',
  'the',
  'teen',
  'generation',
  'that',
  'touches',
  'on',
  'a',
  'very',
  'cool',
  'idea',
  ',',
  'but',
  'presents',
  'it',
  'in',
  'a',
  'very',
  'bad',
  'package',
  '.',
  'which',
  'is',
  'what',
  'makes',
  'this',
  'review',
  'an',
  'even',
  'harder',
  'one',
  'to',
  'write',
  ',',
  'since',
  'i',
  'generally',
  'a

In [58]:
classes = []
def target():
    for id in movie_reviews.fileids():
        if 'pos':
            classes.append(1)
        elif 'neg':
            classes.append(0)
return classes


[]

In [62]:
target()

In [63]:
np.unique(classes)

array([1])

In [35]:
documents[:1]

[(['plot',
   ':',
   'two',
   'teen',
   'couples',
   'go',
   'to',
   'a',
   'church',
   'party',
   ',',
   'drink',
   'and',
   'then',
   'drive',
   '.',
   'they',
   'get',
   'into',
   'an',
   'accident',
   '.',
   'one',
   'of',
   'the',
   'guys',
   'dies',
   ',',
   'but',
   'his',
   'girlfriend',
   'continues',
   'to',
   'see',
   'him',
   'in',
   'her',
   'life',
   ',',
   'and',
   'has',
   'nightmares',
   '.',
   'what',
   "'",
   's',
   'the',
   'deal',
   '?',
   'watch',
   'the',
   'movie',
   'and',
   '"',
   'sorta',
   '"',
   'find',
   'out',
   '.',
   '.',
   '.',
   'critique',
   ':',
   'a',
   'mind',
   '-',
   'fuck',
   'movie',
   'for',
   'the',
   'teen',
   'generation',
   'that',
   'touches',
   'on',
   'a',
   'very',
   'cool',
   'idea',
   ',',
   'but',
   'presents',
   'it',
   'in',
   'a',
   'very',
   'bad',
   'package',
   '.',
   'which',
   'is',
   'what',
   'makes',
   'this',
   'review',
   'an'

In [10]:
len(documents)

2000

In [24]:
len(movie_reviews.fileids())

2000

In [13]:
with open('answer1.txt', 'w') as f:
    f.write(str(len(documents)))

In [95]:
df = pd.DataFrame.from_records(documents, exclude=['docs'], columns = ['docs' , 'cat']) 

In [93]:
df.cat = df.cat.apply(lambda x: 0 if 'neg' else 1)

In [94]:
df

,cat
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0
